In [1]:
import requests as rq
import cx_Oracle as db
import insert as dbi
import datetime as dt
import time

sensors = [
    'http://nosql.hpeixoto.me/api/sensor/3001',
    'http://nosql.hpeixoto.me/api/sensor/3002',
    'http://nosql.hpeixoto.me/api/sensor/3003',
    'http://nosql.hpeixoto.me/api/sensor/3004',
    'http://nosql.hpeixoto.me/api/sensor/3005'
]

# Initializes the tables with the 'tables.sql' file. The file has '!' separating the different statements.
def initTables(conn,cursor):
    fd = open("tables.sql",'r')
    sql = fd.read()
    fd.close()

    commands = sql.split('!')

    for command in commands:
        cursor.execute(command)   

# Deletes all tables
def dropTables(cursor):
    fd = open("drop.sql",'r')
    sql = fd.read()
    fd.close()

    commands = sql.split(";")

    for command in commands:
        cursor.execute(command)

# For each .json datafile, inserts the info correctly in the database
def parseJson(data,conn,cursor):
    careteam = []
    date = dbi.makeDate(data["patient"]["patientbirthdate"],dt)
    dbi.insertPatient(data["patient"]["patientid"],data["patient"]["patientname"],date,data["patient"]["patientage"],cursor)
    dbi.insertService(data["servicecod"],data["servicedesc"],cursor)
    dbi.insertSensor(data["sensorid"],data["sensornum"],data["type_of_sensor"],cursor)
    for medic in data["careteam"]:
        dbi.insertMedic(medic["id"],medic["nome"],cursor)
        careteam.append(medic["id"])
    ct_id = dbi.insertCareteam(careteam,cursor)
    dbi.insertMonitor(data["sensorid"],data["patient"]["patientid"],data["servicecod"],ct_id,data["number_of_sensors"],data["admdate"],data["bed"],data["bodytemp"],data["bloodpress"]["systolic"],data["bloodpress"]["diastolic"],data["bpm"],data["sato2"],data["timestamp"],cursor)

# Uncomment the following line and add your correct Oracle Client directory in case something is breaking
# db.init_oracle_client(lib_dir=r"C:\Users\LENOVO\Desktop\BDNoSQL\instantclient_19_8")

# Assuming that the tablespace and user were created previously
conn = db.connect("sensor", "sensor_api", "localhost:1521/orclpdb1.localdomain")
cursor = conn.cursor()

dropTables(cursor)
initTables(conn,cursor)

while True:
    print("New round of GET requests")
    for sensor in sensors:
        data = rq.get(sensor).json()
        parseJson(data,conn,cursor)
    time.sleep(5)


New round of GET requests
Workteam was added with IDs 39887 28873 332345
1 / 1
Workteam was added with IDs 11244 28873 332345
2 / 2
Workteam was added with IDs 28873 332345 334567
3 / 3
Workteam was added with IDs 28873 332345 332345
4 / 4
Workteam was added with IDs 28873 332345 887433
5 / 5
New round of GET requests
Workteam exists with IDs: 28873 39887 332345
2 / 6
Already Monitored: 101 2020-11-11 10:56:19
Workteam exists with IDs: 11244 28873 332345
3 / 6
Already Monitored: 102 2020-11-11 10:55:37
Workteam exists with IDs: 28873 332345 334567
4 / 6
Already Monitored: 103 2020-11-11 10:55:37
Workteam exists with IDs: 28873 332345 332345
5 / 6
Already Monitored: 104 2020-11-11 10:55:37
Workteam exists with IDs: 28873 332345 887433
6 / 6
Already Monitored: 105 2020-11-11 10:56:02
New round of GET requests
Workteam exists with IDs: 28873 39887 332345
2 / 6
Already Monitored: 101 2020-11-11 10:56:19
Workteam exists with IDs: 11244 28873 332345
3 / 6
Already Monitored: 102 2020-11-11 10

KeyboardInterrupt: 